In [ ]:
import requests
import datetime as dt
from datetime import datetime, timedelta

def check_current_rain(api_key, city="Portland", country_code="US"):
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    
    # Parameters for the API request
    params = {
        "q": f"{city},{country_code}",
        "appid": api_key,
        "units": "imperial"  # For temperature in Fahrenheit
    }
    
    try:
        # Make the API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise exception for HTTP errors
        
        # Parse the JSON response
        data = response.json()
        
        # Check if it's raining
        if "rain" in data:
            print(f"Current weather in {city}: It's raining!")
            print(f"Rain details: {data['rain']}")
            return True
        else:
            weather_description = data["weather"][0]["description"]
            print(f"Current weather in {city}: {weather_description}")
            
            # Check if the weather description contains "rain"
            is_raining = "rain" in weather_description.lower()
            if is_raining:
                print("Rain is detected in the weather description.")
                return True
            
            print("No rain detected.")
            return False
            
    except Exception as e:
        print(f"Error checking current weather: {e}")
        return None


def check_forecast_rain(api_key, city="Portland", country_code="US", days=3):
    base_url = "https://api.openweathermap.org/data/2.5/forecast"
    
    # Parameters for the API request
    params = {
        "q": f"{city},{country_code}",
        "appid": api_key,
        "units": "imperial"  # For temperature in Fahrenheit
    }
    
    try:
        # Make the API request
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise exception for HTTP errors
        
        # Parse the JSON response
        data = response.json()
        
        # Calculate the end date for our forecast period
        today = datetime.now()
        end_date = today + timedelta(days=days)
        
        rain_forecast = False
        rain_times = []
        
        for forecast in data["list"]:
            # Convert timestamp to datetime
            forecast_time = datetime.fromtimestamp(forecast["dt"])
            
            # Skip if beyond our forecast period
            if forecast_time > end_date:
                continue
            
            # Check if it's raining in this forecast period
            weather_desc = forecast["weather"][0]["description"].lower()
            has_rain_obj = "rain" in forecast
            has_rain_desc = "rain" in weather_desc
            
            if has_rain_obj or has_rain_desc:
                rain_forecast = True
                rain_times.append((forecast_time, weather_desc))
        
        # Print the results
        if rain_forecast:
            print(f"\nRain is forecasted in {city} within the next {days} days!")
            print("Rain is expected at the following times:")
            for time, desc in rain_times:
                print(f"  - {time.strftime('%Y-%m-%d %H:%M')} - {desc}")
        else:
            print(f"\nNo rain is forecasted in {city} within the next {days} days.")
        
        return rain_forecast
            
    except Exception as e:
        print(f"Error checking forecast: {e}")
        return None


def main():
    # Replace with your actual OpenWeatherMap API key
    api_key = "xx"
    
    print("Question A: Is it raining in Portland, OR right now?")
    current_rain = check_current_rain(api_key)
    if current_rain is not None:
        answer_a = "Yes" if current_rain else "No"
        print(f"Answer: {answer_a}, it's not currently raining in Portland, OR.\n")
    
    print("Question B: Is it forecasted to be raining in Portland within the next three days?")
    forecast_rain = check_forecast_rain(api_key)
    if forecast_rain is not None:
        answer_b = "Yes" if forecast_rain else "No"
        print(f"Answer: {answer_b}, it's {'not ' if not forecast_rain else ''}forecasted to rain in Portland, OR within the next three days.")

main()

Question A: Is it raining in Portland, OR right now?
Current weather in Portland: scattered clouds
No rain detected.
Answer: No, it's not currently raining in Portland, OR.

Question B: Is it forecasted to be raining in Portland within the next three days?

Rain is forecasted in Portland within the next 3 days!
Rain is expected at the following times:
  - 2025-04-19 05:00 - light rain
Answer: Yes, it's forecasted to rain in Portland, OR within the next three days.
